In [1]:
import pandas as pd
import numpy as np
import socket as sk
import struct as st
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data.csv", index_col = [0])

# Finding the bad guys!

In [3]:
#router's address 
df.loc[df.srcip == '192168241243','srcip'] = '192.168.241.243'
df.loc[df.dstip == '192168241243','dstip'] = '192.168.241.243'

In [4]:
#Converting Hex to Base 10
df.loc[df.dsport == '0xc0a8','dsport'] = '49320'
df.loc[df.dsport == '0x20205321','dsport'] = '538989345'
df.loc[df.dsport == '0xcc09','dsport'] = '52233'
df.loc[df.sport == '0x000b','sport'] = '11'
df.loc[df.sport == '0x000c','sport'] = '12'

In [5]:
#Converting IP to a numerical
df['srcip'] = df.srcip.apply(lambda x: st.unpack('!I', sk.inet_aton(x))[0])
df['dstip'] = df.dstip.apply(lambda x: st.unpack('!I', sk.inet_aton(x))[0])

In [6]:
# Getting rid of unnecessary column
df.drop(columns='attack_cat', inplace = True)

#Handling missing values
df.drop(df.loc[df.dsport == '-'].index, inplace = True)
df.drop(df.loc[df.sport == '-'].index, inplace = True)

In [7]:
# Handling inconsistent data
df.dur = df.dur.str.replace(',','.')
df.ackdat = df.ackdat.str.replace(',','.')
df.loc[df.ct_ftp_cmd == ' ', 'ct_ftp_cmd'] = df.ct_flw_http_mthd.mean()

In [8]:
#Convert objet 
df.sport.astype(int)
df.dsport.astype(int)
df.ackdat.astype(float)
df.dur.astype(float)

0         0.001055
1         0.036133
2         0.001119
3         0.001209
4         0.001169
            ...   
440039         NaN
440040         NaN
440041         NaN
440042         NaN
440043         NaN
Name: dur, Length: 2540011, dtype: float64

In [9]:
#Handling missing values
df.ct_flw_http_mthd.fillna(df.ct_flw_http_mthd.mean(), inplace = True)
df.is_ftp_login.fillna(df.is_ftp_login .mean(), inplace = True)
#df.dur.fillna(df.dur.mean(), inplace = True)
#df.ackdat.fillna(df.ackdat.mean(), inplace = True)

In [10]:
%%time
# Handling categorical data
Encoder = LabelEncoder()
df.proto = Encoder.fit_transform(df.proto)
df.state = Encoder.fit_transform(df.state)
df.service = Encoder.fit_transform(df.service)

CPU times: user 4.87 s, sys: 638 ms, total: 5.51 s
Wall time: 12.5 s


In [11]:
df.to_csv('data-preprocessing.csv')